# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np

In [2]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.8.0
Default GPU Device: /device:GPU:0


First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [3]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [4]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [5]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70],
      dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [6]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

We start with our text encoded as integers in one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`batch_size` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. 

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

In [7]:
def get_batches(arr, batch_size, n_steps):
    '''Create a generator that returns batches of size
       batch_size x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    chars_per_batch = batch_size * n_steps
    n_batches = len(arr)//chars_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * chars_per_batch]
    
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y_temp = arr[:, n+1:n+n_steps+1]
        
        # For the very last batch, y will be one character short at the end of 
        # the sequences which breaks things. To get around this, I'll make an 
        # array of the appropriate size first, of all zeros, then add the targets.
        # This will introduce a small artifact in the last batch, but it won't matter.
        y = np.zeros(x.shape, dtype=x.dtype)
        y[:,:y_temp.shape[1]] = y_temp
        
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [8]:
batches = get_batches(arr=encoded, batch_size=10, n_steps=50)
x, y = next(batches)

In [9]:
print(x.shape, y.shape)

(10, 50) (10, 50)


In [10]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`.

In [11]:
# The input data to/for/of the model
def model_inputs(batch_size, num_steps, lstm_size):    
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    #print(inputs.shape, targets.shape)
    
    # LSTM: Memory cells
    # lstm1 = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    # lstm2 = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    # lstm3 = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    # cell = tf.nn.rnn_cell.MultiRNNCell([lstm1, lstm2, lstm3])
    # lstm = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    # cell = tf.nn.rnn_cell.MultiRNNCell([lstm])

    # GRU: Gated Recurrent Units
    gru = tf.nn.rnn_cell.GRUCell(lstm_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    initial_state = cell.zero_state(batch_size, tf.float32)

    return inputs, targets, cell, initial_state

In [12]:
# RNN generator or sequence generator
def generator(lstm_size, batch_size, num_classes, inputs, initial_state, cell, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
        
        # Input to the RNN
        inputs_onehot = tf.one_hot(indices=inputs, depth=num_classes)

        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        outputs, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_onehot, initial_state=initial_state)
        #print(outputs.shape, final_state.shape)

        seq_output = tf.reshape(outputs, [-1, lstm_size])
        #print(seq_output.shape)

        # with tf.variable_scope('softmax'):
        logits = tf.layers.dense(inputs=seq_output, units=num_classes)
        predictions = tf.nn.softmax(logits)
        #print(predictions.shape)

        # logits for loss calculation, predictions for accuracy/sample generation, and final_state for feedbackloop
        return logits, predictions, final_state

In [13]:
def model_loss(num_classes, lstm_size, batch_size, inputs, targets, cell, initial_state):
    # Build rnncell and initilize it
    logits, predictions, final_state = generator(inputs=inputs, cell=cell, initial_state=initial_state,
                                                 batch_size=batch_size, lstm_size=lstm_size, 
                                                 num_classes=num_classes)

    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    targets_one_hot = tf.one_hot(indices=targets, depth=num_classes)
    labels = tf.reshape(targets_one_hot, logits.get_shape())
    #print(targets.shape, targets_one_hot.shape, labels.shape)

    # Softmax cross entropy loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    
    # Loss for updating, predictions for accuracy and final state for the next bacth
    return loss, predictions, final_state

In [14]:
def model_opt(loss, learning_rate):
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    #print(tvars)
    
    # grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, tvars), clip_norm=5) # usually around 1-5
    grads=tf.gradients(loss, tvars)
    #print(grads)
    
    train_op = tf.train.AdamOptimizer(learning_rate)
    opt = train_op.apply_gradients(grads_and_vars=zip(grads, tvars))
    return opt

In [15]:
class Model:
    def __init__(self, batch_size, num_steps, lstm_size, num_classes, learning_rate):
        
        # The data of the model and the RNNcell in the model and initialize it
        #self.cell, self.initial_state = model_rnncell(batch_size=batch_size, lstm_size=lstm_size)
        self.inputs, self.targets, self.cell, self.initial_state = model_inputs(batch_size=batch_size,
                                                                                lstm_size=lstm_size, 
                                                                                num_steps=num_steps)
        
        # The loss of the model
        self.loss, self.predictions, self.final_state = model_loss(
            batch_size=batch_size, lstm_size=lstm_size, num_classes=num_classes,
            inputs=self.inputs, targets=self.targets, cell=self.cell, initial_state=self.initial_state)
        
        # The update/training of the model
        self.opt = model_opt(learning_rate=learning_rate, loss=self.loss)

In [16]:
# Hyperparameters for training
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
# num_layers = 2        # Number of LSTM layers
learning_rate = 0.001   # Learning rate
epochs = 100            # Epochs/episodes of training/update
num_classes=len(vocab)  # number of classes of letter for input and output/target letters

In [17]:
# Reset/init the graph/session
tf.reset_default_graph()

# Init the model
model = Model(batch_size=batch_size, learning_rate=learning_rate, lstm_size=lstm_size, num_classes=num_classes,
              num_steps=num_steps)

In [18]:
# Save and restore the trained model
saver = tf.train.Saver()

# training losses for plotting
train_losses = []

# A tf session for computational graph (has to be reset)
with tf.Session() as sess:
    
    # Initialize the graph variables/parameters in this session
    sess.run(tf.global_variables_initializer())
    
    # # Use the line below to load a checkpoint and resume training
    # saver.restore(sess, 'checkpoints/model.ckpt')
    #saver.restore(sess, tf.train.latest_checkpoint(checkpoint_dir='checkpoints/'))

    # Training epochs/episodes
    loss_batch = []
    for e in range(epochs):
        
        # Training batches/steps
        initial_state = sess.run(model.initial_state)
        for inputs, targets in get_batches(encoded, batch_size, num_steps):
            # Feeding the batches into the model
            feed_dict = {model.inputs: inputs, 
                         model.targets: targets, 
                         model.initial_state: initial_state}
            loss, final_state, _ = sess.run([model.loss, 
                                             model.final_state, 
                                             model.opt], feed_dict)
            # Feedback loop or recurrence/recursion/lateral connections
            initial_state = final_state
            
            # Save the batch losses
            loss_batch.append(loss)

        # Printing out the loss and the time of training for every epoch/episode
        print('Epoch:{}/{}'.format(e+1, epochs),
              'Training loss:{:.4f}'.format(np.mean(loss_batch)))
        
        # Saving the mean/average training loss batches for plotting
        train_losses.append(np.mean(loss_batch))
        
    # Save the trained model
    #saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    saver.save(sess, "checkpoints/model.ckpt")

Epoch:1/100 Training loss:2.6269
Epoch:2/100 Training loss:2.3145
Epoch:3/100 Training loss:2.1288
Epoch:4/100 Training loss:1.9972
Epoch:5/100 Training loss:1.8977
Epoch:6/100 Training loss:1.8192
Epoch:7/100 Training loss:1.7543
Epoch:8/100 Training loss:1.6990
Epoch:9/100 Training loss:1.6524
Epoch:10/100 Training loss:1.6113
Epoch:11/100 Training loss:1.5751
Epoch:12/100 Training loss:1.5429
Epoch:13/100 Training loss:1.5139
Epoch:14/100 Training loss:1.4876
Epoch:15/100 Training loss:1.4637
Epoch:16/100 Training loss:1.4417
Epoch:17/100 Training loss:1.4215
Epoch:18/100 Training loss:1.4028
Epoch:19/100 Training loss:1.3853
Epoch:20/100 Training loss:1.3690
Epoch:21/100 Training loss:1.3537
Epoch:22/100 Training loss:1.3394
Epoch:23/100 Training loss:1.3260
Epoch:24/100 Training loss:1.3132
Epoch:25/100 Training loss:1.3012
Epoch:26/100 Training loss:1.2897
Epoch:27/100 Training loss:1.2789
Epoch:28/100 Training loss:1.2687
Epoch:29/100 Training loss:1.2590
Epoch:30/100 Training l

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [19]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

Here, pass in the path to a checkpoint and sample from the network.

In [20]:
tf.train.latest_checkpoint('checkpoints/')

'checkpoints/model.ckpt'

In [21]:
# Hyperparameters for testing/ sequence prediction
batch_size = 1          # Sequences per batch
num_steps = 1           # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
# num_layers = 2        # Number of LSTM layers
learning_rate = 0.001   # Learning rate
# epochs = 1              # Epochs/episodes of training/update
num_classes=len(vocab)  # number of classes of letter for input and output/target letters

In [22]:
# Input for the sampling predictor
checkpoint = tf.train.latest_checkpoint('checkpoints')
n_samples=100 
lstm_size=lstm_size
vocab_size=len(vocab)
prime="Far"

In [23]:
# Reset/init the graph/session
tf.reset_default_graph()

# Init the model
model = Model(learning_rate=learning_rate, lstm_size=lstm_size, num_classes=num_classes,
              batch_size=1, num_steps=1)

In [24]:
# Initial sample input
samples = [c for c in prime]

# To restore/load the model
saver = tf.train.Saver()

# A session for computational graph
with tf.Session() as sess:

    # Load/restore the trained model
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints/'))
    #saver.restore(sess, 'checkpoints/model.ckpt')
    saver.restore(sess, checkpoint)

    # Testing
    initial_state = sess.run(model.initial_state)
    for c in prime:
        inputs = np.zeros((1, 1))
        inputs[0,0] = vocab_to_int[c]

        # Model feed and fetch
        feed_dict = {model.inputs: inputs, 
                     model.initial_state: initial_state}
        predictions, final_state = sess.run([model.predictions, 
                                             model.final_state], feed_dict)

        # Feedback/recuurence/recursion/lateral connection
        initial_state = final_state

    c = pick_top_n(preds=predictions, vocab_size=len(vocab))
    samples.append(int_to_vocab[c])

    for i in range(n_samples):
        inputs[0,0] = c

        # Model feed and fetch
        feed_dict = {model.inputs: inputs, 
                     model.initial_state: initial_state}
        predictions, final_state = sess.run([model.predictions, 
                                             model.final_state], feed_dict)

        # Feedback/recuurence/recursion/lateral connection
        initial_state = final_state

        c = pick_top_n(preds=predictions, vocab_size=len(vocab))
        samples.append(int_to_vocab[c])

print(''.join(samples))

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
Farrown folding
sight, but he called to the still son. He drove out a will gellov with a
clover eyes gro
